A CNN model of discrimination using CIFAR-10, I made a reference to http://machinelearningmastery.com/object-recognition-convolutional-neural-networks-keras-deep-learning-library/ of the model, and changed some parameter such as activate function, dropout rate and learning rate after checking VGG16, and http://localhost:8888/notebooks/GAN_on_Keras_by_osh.ipynb, a GAN model's discriminative part done by Tim O'Shea.

In [1]:
# Simple CNN model for CIFAR-10
%matplotlib inline
import numpy
import os,random
os.environ["KERAS_BACKEND"] = "tensorflow"
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('tf')

Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [3]:
# load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [4]:
# normalize inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [5]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [6]:
# Create the model
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(3, 32, 32), border_mode='same', activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Convolution2D(32, 3, 3, activation='linear', border_mode='same', W_constraint=maxnorm(3)))
model.add(LeakyReLU(alpha=0.2)) 
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
# Compile model
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 3, 32, 32)     9248        convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 3, 32, 32)     0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 3, 32, 32)     9248        dropout_1[0][0]                  
____________________________________________________________________________________________________
leakyrelu_1 (LeakyReLU)          (None, 3, 32, 32)     0           convolution2d_2[0][0]            
___________________________________________________________________________________________

In [ ]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=epochs, batch_size=32)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 176s - loss: 1.8410 - acc: 0.3311 - val_loss: 1.5414 - val_acc: 0.4416
Epoch 2/25
50000/50000 [==============================] - 161s - loss: 1.5581 - acc: 0.4410 - val_loss: 1.4194 - val_acc: 0.4887
Epoch 3/25
50000/50000 [==============================] - 458s - loss: 1.4544 - acc: 0.4777 - val_loss: 1.3826 - val_acc: 0.5043
Epoch 4/25
50000/50000 [==============================] - 179s - loss: 1.3368 - acc: 0.5215 - val_loss: 1.2703 - val_acc: 0.5438
Epoch 6/25
50000/50000 [==============================] - 179s - loss: 1.3003 - acc: 0.5324 - val_loss: 1.2489 - val_acc: 0.5537
Epoch 7/25
50000/50000 [==============================] - 583s - loss: 1.2125 - acc: 0.5667 - val_loss: 1.1831 - val_acc: 0.5771
Epoch 11/25
50000/50000 [==============================] - 489s - loss: 1.2006 - acc: 0.5730 - val_loss: 1.1604 - val_acc: 0.5871
Epoch 12/25
50000/50000 [=====================